Make a file with a groupby with just the census tracts. I have done a Hadoop job to write all the rows, and they need to be processed to be nicer.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import os
import ast

In [3]:
path = 'Ratios/'
files = os.listdir(path)

In [18]:
rows = []
with open('Ratios/part-00000', 'r') as f:
    for line in f:
        key, count = line.split('\t')
        direction, tract = ast.literal_eval(key)
        row = [direction, int(tract), int(count)]
        rows.append(row)

This works.

In [44]:
rows = []
for file in files:
    if file.startswith('part'):
        with open(path + file, 'r') as f:
            for line in f:
                key, count = line.split('\t')
                direction, tract = ast.literal_eval(key)
                row = [direction, tract, int(count)]
                rows.append(row)

In [45]:
df = pd.DataFrame(rows)

In [46]:
df.head()

,0,1,2
0,from,17031010701,44
1,from,17031020802,197
2,from,17031032100,17240
3,from,17031062100,24428
4,from,17031071200,25692


In [47]:
df.columns = ['direction', 'census_tract', 'count']

In [48]:
total_from = df[df['direction'] == 'from'].sum()['count']
total_to = df[df['direction'] == 'to'].sum()['count']

In [49]:
df = df.set_index('census_tract').pivot_table(values='count',
                                         index=['census_tract'],
                                        columns=['direction']).fillna(0)

In [50]:
df['ratio_from'] = df['from'] / total_from
df['ratio_to'] = df['to'] / total_to

In [51]:
df.head()

direction,from,to,ratio_from,ratio_to
census_tract,,,,
17031010100,46.0,0.0,0.000002,0.000000e+00
17031010201,55.0,3.0,0.000003,1.763170e-07
17031010202,46.0,2.0,0.000002,1.175447e-07
17031010300,524.0,63.0,0.000028,3.702657e-06
17031010400,1134.0,38.0,0.000061,2.233349e-06


In [52]:
df.to_csv('Data/Ratios.csv')

In [57]:
df.loc['17031838300']

direction
from          13991.000000
to             3344.000000
ratio_from        0.000747
ratio_to          0.000197
Name: 17031838300, dtype: float64